# 03 - Federated Learning


Define the available types of federated learning.

- 'STRATIFIED': Stratified sampling of the data. The data is split into a number of shards, and each shard is assigned to a client. The data is split in a stratified manner, meaning that the distribution of the labels is approximately the same in each shard.
- 'LEAVE_ONE_OUT' - Each client is assigned a shard of data, each shard is missing one of the attack labels. Other clients in the network are exposed to the attack label, but the specific client is not. This demonstrates the ability of federated learning to protect against unknown attacks.
- 'ONE_CLASS' - Each client is assigned a shard of data, each shard contains only one of the attack labels.
- 'HALF_BENIGN' - Half of the clients are exposed to Benign data only, the other half are exposed to all data.


In [1]:
### THIS SECTION NEEDS TO BE SET TO DETERMINE WHICH CONFIGURATION METHOD TO UTILISE

SPLIT_AVAILABLE_METHODS = ['STRATIFIED','LEAVE_ONE_OUT', 'ONE_CLASS', 'HALF_BENIGN' ]
METHOD = 'ONE_CLASS'
NUM_OF_STRATIFIED_CLIENTS = 10  # only applies to stratified method
NUM_OF_ROUNDS = 10              # Number of FL rounds


The above test method in conjunction with the below classification selection will determine the number of clients.

EG:
`STRATIFIED` with:

- `ALL TYPES` - Results in `NUM_OF_STRATIFIED_CLIENTS` clients. Each client will have a stratified sample of the data.

`LEAVE_ONE_OUT` with:

- `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 32 attack labels.
- `group_classifier` - Results in 7 clients. Each client will have benign traffic and 6 attack groups.
- `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels.

`ONE_CLASS` with:

- `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 1 attack label.
- `group_classifier` - Results in 7 clients. Each client will have benign traffic and 1 attack groups.
- `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS LEAVE_ONE_OUT for binary classifier

`HALF_BENIGN` with:

- `individual_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 33 malicious attack labels.
- `group_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 7 malicious attack groups.
- `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS LEAVE_ONE_OUT for binary classifier


In [2]:

individual_classifier = False
group_classifier = False
binary_classifier = True

Include the defines for the dataframe columns and the attack labels and their mappings


In [3]:
from enum import Enum
from includes import *

## Imports


In [4]:
%%capture
%pip install flwr[simulation] torch torchvision matplotlib sklearn openml

In [5]:
import os
import pandas as pd
import numpy as np
import flwr as fl
from tqdm import tqdm
import warnings
#warnings.filterwarnings('ignore')

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split


2025-06-29 03:15:04.085108: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/user/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/user/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-29 03:15:14,718	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook

In [6]:
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 1.18.0
numpy 1.26.4
torch 2.2.2
Training on cpu


## Load the Dataset


In [7]:
DATASET_DIRECTORY = './Dataset/'

## Training data


Either read the training pickle file if it exists, or process the dataset from scratch.


In [8]:
if os.path.isfile('training_data.pkl'):
    print("File exists, loading data...")
    train_df = pd.read_pickle('training_data.pkl')
    print("Training data loaded from pickle file.")
else:
    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    # Print the number of files in each set
    print('Training sets: {}'.format(len(training_sets)))
    print('Test sets: {}'.format(len(test_sets)))

    # Concatenate all training sets into one dataframe
    dfs = []
    print("Reading training data...")
    for train_set in tqdm(training_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + train_set)
        dfs.append(df_new)
    train_df = pd.concat(dfs, ignore_index=True)

    # Complete training data set size
    print("Complete training data size: {}".format(train_df.shape))

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    train_df['label'] = train_df['label'].map(dict_34_classes)
    
    # Percentage of original training data to use.
    TRAIN_SIZE = 0.1
    
    print(f"Splitting the data into {TRAIN_SIZE*100}%")
    
    X_train, X_test, y_train, y_test = train_test_split(train_df[X_columns], train_df[y_column], test_size= (1 - TRAIN_SIZE), random_state=42, stratify=train_df[y_column])

    # Recombine X_train, and y_train into a dataframe
    train_df = pd.concat([X_train, y_train], axis=1)

    # Clean up unused variables

    del X_train, y_train, X_test, y_test
    
    print("Writing training data to pickle file...")
    train_df.to_pickle('training_data.pkl')

print("Training data size: {}".format(train_df.shape))


Training sets: 132
Test sets: 34
Reading training data...


100%|██████████| 132/132 [03:16<00:00,  1.49s/it]


Complete training data size: (35613668, 47)
Splitting the data into 10.0%
Writing training data to pickle file...
Training data size: (3561366, 47)


In [9]:
# show the unique values counts in the label column for train_df
print("Counts of attacks in train_df:")
print(train_df['label'].value_counts())

Counts of attacks in train_df:
label
6     7008
4     5297
5     4411
2     4058
3     3997
1     3970
7     3499
13    3246
15    2596
14    1959
0     1080
17     974
19     897
18     736
10     451
26     305
8      289
9      288
25     175
24     135
21      94
22      83
16      76
23      35
12      30
11      21
33      12
31       6
27       6
32       5
28       4
29       3
20       2
30       1
Name: count, dtype: int64


In [10]:
train_df.head()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
296519,0.000000,54.00,6.00,64.00,1.999744,1.999744,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.306852e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,5
55514,0.000000,0.00,1.00,64.00,13.745034,13.745034,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.314963e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,6
250771,0.000000,0.00,47.00,64.00,10.150477,10.150477,0.0,0.0,0.0,0.0,...,0.000000,592.00,8.368107e+07,9.5,34.409301,0.000000,0.000000,0.00,141.55,17
392529,0.252713,20998.73,16.78,64.00,4164.637934,4164.637934,0.0,0.0,0.0,0.0,...,22.137532,55.49,8.312387e+07,9.5,10.596516,31.347346,3345.013538,0.16,141.55,4
230400,0.000000,53.46,5.94,63.36,12.444320,12.444320,0.0,0.0,1.0,0.0,...,0.185955,54.06,8.336185e+07,9.5,10.397389,0.263434,0.292861,0.12,141.55,7


---

## Test Data

Concat the test data into a single dataframe


In [11]:
testing_data_pickle_file = 'testing_data.pkl'

if os.path.isfile(testing_data_pickle_file):
    print(f"File {testing_data_pickle_file} exists, loading data...")
    test_df = pd.read_pickle(testing_data_pickle_file)
    print("Test data loaded from pickle file.")

else:
    print(f"File {testing_data_pickle_file} does not exist, constructing data...")

    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    # Print the number of files in each set
    print('Test sets: {}'.format(len(test_sets)))
    
    dfs = []
    print("Reading test data...")
    for test_set in tqdm(test_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + test_set)
        dfs.append(df_new)
    test_df = pd.concat(dfs, ignore_index=True)

    test_df['label'] = test_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print(f"Writing test data to pickle file {testing_data_pickle_file}...")
    test_df.to_pickle(testing_data_pickle_file)

print("Testing data size: {}".format(test_df.shape))

File testing_data.pkl exists, loading data...
Test data loaded from pickle file.
Testing data size: (275258, 47)


In [12]:
print("Number of rows in train_df: {}".format(len(train_df)))
print("Number of rows in test_df: {}".format(len(test_df)))

train_size = len(train_df)
test_size = len(test_df)

Number of rows in train_df: 45749
Number of rows in test_df: 275258


---

# Scale the test and train data


### Scale the training data input features


In [13]:
scaler = StandardScaler()
train_df[X_columns] = scaler.fit_transform(train_df[X_columns])

### Scale the testing data input features


In [14]:
test_df[X_columns] = scaler.fit_transform(test_df[X_columns])

---

# Define the classification problem - (2 classes, 8 classes or 34 classes)

Change the following cell to select the classification type

If the METHOD == STRATIFIED, then we can use any classifier
If the METHOD == ATTACK_GROUP then we must use Group Classifier.


In [15]:

class_size_map = {2: "Binary", 8: "Group", 34: "Individual"}

if group_classifier:
    print("Group 8 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_7_classes values
    test_df['label'] = test_df['label'].map(dict_8_classes)
    train_df['label'] = train_df['label'].map(dict_8_classes)
    class_size = "8"      
    
elif binary_classifier:
    print("Binary 2 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_2_classes values
    test_df['label'] = test_df['label'].map(dict_2_classes)
    train_df['label'] = train_df['label'].map(dict_2_classes)
    class_size = "2"

else:
    print ("Individual 34 Class classifier... - No adjustments to labels in test and train dataframes")
    class_size = "34"


Binary 2 Class Classifier... - Adjusting labels in test and train dataframes


---

# Split the Training Data into partitions for the Federated Learning clients depending on the test required


In [16]:
from sklearn.model_selection import StratifiedKFold

# Define fl_X_train and fl_y_train
fl_X_train = []
fl_y_train = []

client_df = pd.DataFrame()

if METHOD == 'STRATIFIED':
    print(f"{Colours.YELLOW.value}STRATIFIED METHOD{Colours.NORMAL.value} with {class_size} class classifier")
   
    skf = StratifiedKFold(n_splits=NUM_OF_STRATIFIED_CLIENTS, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(train_df[X_columns], train_df[y_column]):
        fl_X_train.append(train_df[X_columns].iloc[test_index])
        fl_y_train.append(train_df[y_column].iloc[test_index])

elif METHOD == 'LEAVE_ONE_OUT':
    print(f"{Colours.YELLOW.value}LEAVE_ONE_OUT METHOD{Colours.NORMAL.value} with {class_size} class classifier")

    if individual_classifier or group_classifier:
       
        num_splits = int(class_size) - 1
    else:
        
        num_splits = 10

    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if binary_classifier:
            print(f"i: {i} = i % 2 = {i % 2}")
            if i % 2 == 0:
                print("Benign only")
                client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 1]], ignore_index=True)
                fl_X_train.append(client_df[X_columns])
                fl_y_train.append(client_df[y_column])
            else:
                print("Both")
                fl_X_train.append(train_df[X_columns].iloc[test_index])
                fl_y_train.append(train_df[y_column].iloc[test_index])
        else:
            client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i+1]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])

elif METHOD == 'ONE_CLASS':
    print(f"{Colours.YELLOW.value}ONE_CLASS METHOD{Colours.NORMAL.value} with {class_size} class classifier")
    
    
    if individual_classifier or group_classifier:
        num_splits = int(class_size) - 1
    else:
        num_splits = 10

    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if binary_classifier:
            print(f"i: {i} = i % 2 = {i % 2}")
            if i % 2 == 0:
                print("Benign only")
                client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 1]], ignore_index=True)
                fl_X_train.append(client_df[X_columns])
                fl_y_train.append(client_df[y_column])
            else:
                print("Both")

                fl_X_train.append(train_df[X_columns].iloc[test_index])
                fl_y_train.append(train_df[y_column].iloc[test_index])
        else:
            client_df = pd.concat([train_df.iloc[test_index][(train_df[y_column] == 0) | (train_df[y_column] == i+1)]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])

elif METHOD == 'HALF_BENIGN':
    print(f"{Colours.YELLOW.value}HALF_BENIGN METHOD{Colours.NORMAL.value} with {class_size} class classifier")

    num_splits = 10

    # Split into 10 client data
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    # For i % 2 == 0, add only benign data
    # For i % 2 == 1, add all data
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if i % 2 == 0:
            print("Benign only")
            client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] == 0]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])
        else:
            print("All Classes")
            fl_X_train.append(train_df[X_columns].iloc[test_index])
            fl_y_train.append(train_df[y_column].iloc[test_index])
else:
    print(f"{Colours.RED.value}ERROR: Method {METHOD} not recognised{Colours.NORMAL.value}")
    exit()



ONE_CLASS METHOD with 2 class classifier
i: 0 = i % 2 = 0
Benign only
i: 1 = i % 2 = 1
Both
i: 2 = i % 2 = 0
Benign only
i: 3 = i % 2 = 1
Both
i: 4 = i % 2 = 0
Benign only
i: 5 = i % 2 = 1
Both
i: 6 = i % 2 = 0
Benign only
i: 7 = i % 2 = 1
Both
i: 8 = i % 2 = 0
Benign only
i: 9 = i % 2 = 1
Both


/var/folders/q0/6fymk865243fx1n2trkz4zl40000gn/T/ipykernel_11908/2810336824.py:62: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 1]], ignore_index=True)
/var/folders/q0/6fymk865243fx1n2trkz4zl40000gn/T/ipykernel_11908/2810336824.py:62: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 1]], ignore_index=True)
/var/folders/q0/6fymk865243fx1n2trkz4zl40000gn/T/ipykernel_11908/2810336824.py:62: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 1]], ignore_index=True)
/var/folders/q0/6fymk865243fx1n2trkz4zl40000gn/T/ipykernel_11908/2810336824.py:62: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 

In [17]:
NUM_OF_CLIENTS = len(fl_X_train)

for i in range(len(fl_X_train)):
    (f"Client ID: {i}")
    print(f"fl_X_train[{i}].shape: {fl_X_train[i].shape}")  
    print(f"fl_y_train[{i}].value_counts():\n{fl_y_train[i].value_counts()}")
    print(f"fl_y_train[{i}].unique(): {fl_y_train[i].unique()}\n")

print(f"fl_X_train[0].equals(fl_X_train[1]): {fl_X_train[0].equals(fl_X_train[1])}")

fl_X_train[0].shape: (108, 46)
fl_y_train[0].value_counts():
label
0    108
Name: count, dtype: int64
fl_y_train[0].unique(): [0]

fl_X_train[1].shape: (4575, 46)
fl_y_train[1].value_counts():
label
1    4467
0     108
Name: count, dtype: int64
fl_y_train[1].unique(): [1 0]

fl_X_train[2].shape: (108, 46)
fl_y_train[2].value_counts():
label
0    108
Name: count, dtype: int64
fl_y_train[2].unique(): [0]

fl_X_train[3].shape: (4575, 46)
fl_y_train[3].value_counts():
label
1    4467
0     108
Name: count, dtype: int64
fl_y_train[3].unique(): [1 0]

fl_X_train[4].shape: (108, 46)
fl_y_train[4].value_counts():
label
0    108
Name: count, dtype: int64
fl_y_train[4].unique(): [0]

fl_X_train[5].shape: (4575, 46)
fl_y_train[5].value_counts():
label
1    4467
0     108
Name: count, dtype: int64
fl_y_train[5].unique(): [1 0]

fl_X_train[6].shape: (108, 46)
fl_y_train[6].value_counts():
label
0    108
Name: count, dtype: int64
fl_y_train[6].unique(): [0]

fl_X_train[7].shape: (4575, 46)
fl_y_trai

Prepare an output directory where we can store the results of the federated learning


In [18]:

if not os.path.exists("Output"):
    os.makedirs("Output")

sub_dir_name = f"train_size-{train_size}"

# if sub_dir_name does not exist, create it
if not os.path.exists(f"Output/{sub_dir_name}"):
    os.makedirs(f"Output/{sub_dir_name}")

test_directory_name = f"{METHOD}_Classifier-{class_size}_Clients-{NUM_OF_CLIENTS}"

if not os.path.exists(f"Output/{sub_dir_name}/{test_directory_name}"):
    os.makedirs(f"Output/{sub_dir_name}/{test_directory_name}")

# Ensure the directory is empty
for file in os.listdir(f"Output/{sub_dir_name}/{test_directory_name}"):
    file_path = os.path.join(f"Output/{sub_dir_name}/{test_directory_name}", file)
    if os.path.isfile(file_path):
        os.unlink(file_path)

# Original training size is the sum of all the fl_X_train sizes
original_training_size = 0
for i in range(len(fl_X_train)):
    original_training_size += fl_X_train[i].shape[0]

# Write this same info to the output directory/Class Split Info.txt
with open(f"Output/{sub_dir_name}/{test_directory_name}/Class Split Info.txt", "w") as f:
    for i in range(len(fl_X_train)):
        f.write(f"Client ID: {i}\n")
        f.write(f"fl_X_train.shape: {fl_X_train[i].shape}\n")
        f.write(f"Training data used {original_training_size}")
        f.write(f"fl_y_train.value_counts():\n{fl_y_train[i].value_counts()}\n")
        f.write(f"fl_y_train.unique(): {fl_y_train[i].unique()}\n\n")

### Convert the training dataset


In [19]:
# Convert the testing daya to X_test and y_test ndarrays
X_test = test_df[X_columns].to_numpy()
y_test = test_df[y_column].to_numpy()

In [20]:
num_unique_classes = len(train_df[y_column].unique())

train_df_shape = train_df.shape
test_df_shape = test_df.shape

# We are now done with the train_df and test_df dataframes, so we can delete them to free up memory
del train_df
del test_df
del client_df

---

### Data check


In [21]:
print("NUM_CLIENTS:", NUM_OF_CLIENTS)

print("NUM_ROUNDS:", NUM_OF_ROUNDS)
print()


print("Original training size: {}".format(original_training_size))


print("Checking training data split groups")
for i in range(len(fl_X_train)):
    print(i, ":", "X Shape", fl_X_train[i].shape, "Y Shape", fl_y_train[i].shape)


# Print the sizes of X_test and y_test
print("\nChecking testing data")
print("X_test size: {}".format(X_test.shape))
print("y_test size: {}".format(y_test.shape))

print("\nDeploy Simulation")

NUM_CLIENTS: 10
NUM_ROUNDS: 10

Original training size: 23414
Checking training data split groups
0 : X Shape (108, 46) Y Shape (108,)
1 : X Shape (4575, 46) Y Shape (4575,)
2 : X Shape (108, 46) Y Shape (108,)
3 : X Shape (4575, 46) Y Shape (4575,)
4 : X Shape (108, 46) Y Shape (108,)
5 : X Shape (4575, 46) Y Shape (4575,)
6 : X Shape (108, 46) Y Shape (108,)
7 : X Shape (4575, 46) Y Shape (4575,)
8 : X Shape (108, 46) Y Shape (108,)
9 : X Shape (4574, 46) Y Shape (4574,)

Checking testing data
X_test size: (275258, 46)
y_test size: (275258,)

Deploy Simulation


---

# Federated Learning

## Import the libraries and print the versions


In [22]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout



Define the Client and Server code


In [23]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

print('scikit-learn {}.'.format(sklearn.__version__))
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from flwr.common import Context
from tensorflow.keras.layers import Input

import datetime

client_evaluations = []

class NumpyFlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, train_data, train_labels):
        self.model = model
        self.cid = cid
        self.train_data = train_data
        self.train_labels = train_labels

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Training...")
        self.model.fit(self.train_data, self.train_labels, epochs=5, batch_size=32)
        print ("Client ", self.cid, "Training complete...")
        return self.model.get_weights(), len(self.train_data), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Evaluating...")
        loss, accuracy = self.model.evaluate(self.train_data, self.train_labels, batch_size=32)
        print(f"{Colours.YELLOW.value}Client {self.cid} evaluation complete - Accuracy: {accuracy:.6f}, Loss: {loss:.6f}{Colours.NORMAL.value}")

        # Write the same message to the "Output/{cid}_Evaluation.txt" file
        with open(f"Output/{sub_dir_name}/{test_directory_name}/{self.cid}_Evaluation.txt", "a") as f:
            f.write(f"{datetime.datetime.now()} - Client {self.cid} evaluation complete - Accuracy: {accuracy:.6f}, Loss: {loss:.6f}\n")

            # Close the file
            f.close()

        return loss, len(self.train_data), {"accuracy": accuracy}
    
    def predict(self, incoming):
        prediction = np.argmax( self.model.predict(incoming) ,axis=1)
        return prediction

def client_fn(cid: str) -> NumpyFlowerClient:
# def client_fn(context: Context) -> fl.client.Client:
#     cid = context.client_id
    """Create a Flower client representing a single organization."""

    # Load model
    #model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    print ("Client ID:", cid)

    model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(50, activation='relu'),  
      Dense(25, activation='relu'),  
      Dense(num_unique_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

   
    partition_id = int(cid)
    X_train_c = fl_X_train[partition_id]
    y_train_c = fl_y_train[partition_id]

    # Create a  single Flower client representing a single organization
    return NumpyFlowerClient(cid, model, X_train_c, y_train_c).to_client()


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
eval_count = 0

def get_evaluate_fn(server_model):
    global eval_count
    """Return an evaluation function for server-side evaluation."""
    # The `evaluate` function will be called after every round
    
    
    def evaluate(server_round, parameters, config):
        global eval_count
        
        # Update model with the latest parameters
        server_model.set_weights(parameters)
        print (f"Server Evaluating... Evaluation Count:{eval_count}")
        loss, accuracy = server_model.evaluate(X_test, y_test)
        
        y_pred = server_model.predict(X_test)
        print ("Prediction: ", y_pred, y_pred.shape)
        #cmatrix = confusion_matrix(y_test, np.rint(y_pred))
        #print ("confusion_matrix:", cmatrix, cmatrix.shape)
                        
        print(f"{Colours.YELLOW.value}Server evaluation complete - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}{Colours.NORMAL.value}")

        # Write the same message to the "Output/Server_Evaluation.txt" file
        with open(f"Output/{sub_dir_name}/{test_directory_name}/Server_Evaluation.txt", "a") as f:
            f.write(f"{datetime.datetime.now()} - {server_round} : Server evaluation complete - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}\n")

            # Close the file
            f.close()
        
        np.save("y_pred-" + str(eval_count) + ".npy", y_pred)
        #np.save("cmatrix-" + str(eval_count) + ".npy", cmatrix)
        eval_count = eval_count + 1
        
        return loss, {"accuracy": accuracy}
    return evaluate



server_model = Sequential([
    #Flatten(input_shape=(79,1)),
    # Flatten(input_shape=(fl_X_train[0].shape[1], 1)),
    Input(shape=(fl_X_train[0].shape[1], 1)),
    Flatten(),
    Dense(50, activation='relu'),  
    Dense(25, activation='relu'),  
    Dense(num_unique_classes, activation='softmax')
])


server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2, #10,
        min_evaluate_clients=2, #5,
        min_available_clients=2, #10,
        evaluate_fn=get_evaluate_fn(server_model),
        #evaluate_metrics_aggregation_fn=weighted_average,
)

scikit-learn 1.6.1.
flwr 1.18.0
numpy 1.26.4
tf 2.16.2


In [24]:
import ray

try:
    print(f"Ray version: {ray.__version__}")
except AttributeError:
    print("Ray is installed, but __version__ attribute is not found. This is unusual.")
except ImportError:
    print("Ray is not installed in this environment.")

Ray version: 2.31.0


In [25]:
%%time
print (f"{Colours.YELLOW.value}\nDeploy simulation... Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier")
print (f"Number of Clients = {NUM_OF_CLIENTS}\n")
print (f"Writing output to: {sub_dir_name}/{test_directory_name}\n{Colours.NORMAL.value}")

with open(f"Output/{sub_dir_name}/{test_directory_name}/Run_details.txt", "a") as f:
    f.write(f"{datetime.datetime.now()} - Deploy simulation... Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier\n")
    f.write(f"{datetime.datetime.now()} - Number of Clients = {NUM_OF_CLIENTS}\n")

    # Write Original train_df size
    f.write(f"{datetime.datetime.now()} - Original train_df size: {train_df_shape}\n")

    # Write the training data split groups
    for i in range(len(fl_X_train)):
        f.write(f"{datetime.datetime.now()} - {i}: X Shape {fl_X_train[i].shape}, Y Shape {fl_y_train[i].shape}\n")

    # Write the testing data
    f.write(f"{datetime.datetime.now()} - X_test size: {X_test.shape}\n")
    f.write(f"{datetime.datetime.now()} - y_test size: {y_test.shape}\n")
    
# close the file
f.close()

start_time = datetime.datetime.now()

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    strategy=strategy,
)

end_time = datetime.datetime.now()
print("Total time taken: ", end_time - start_time)

print (f"{Colours.YELLOW.value} SIMULATION COMPLETE. Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier")
print (f"Number of Clients = {NUM_OF_CLIENTS}{Colours.NORMAL.value}\n")

# Output the same information to the Output/Run_details.txt file
with open(f"Output/{sub_dir_name}/{test_directory_name}/Run_details.txt", "a") as f:
    f.write(f"{datetime.datetime.now()} - SIMULATION COMPLETE. Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier\n")
    f.write(f"{datetime.datetime.now()} - Total time taken: {end_time - start_time}\n")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout



Deploy simulation... Method = ONE_CLASS - Binary (2) Classifier
Number of Clients = 10

Writing output to: train_size-45749/ONE_CLASS_Classifier-2_Clients-10



2025-06-29 02:40:29,840	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 8.0, 'memory': 5993289319.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 2147483648.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 8 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(raylet) /Users/user/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
(raylet)   warnings.warn(
(

(ClientAppActor pid=11974) Client ID: 2
Server Evaluating... Evaluation Count:0


/Users/user/Library/Python/3.9/lib/python/site-packages/tensorflow/python/framework/tensor_util.py:522: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 2147483648 to int32 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  nparray = np.array(values, dtype=np_dt)


...

Prediction:  [[7.6157952e-10 9.9999994e-01]
 [2.2308679e-08 9.9999994e-01]
 [9.2948133e-10 9.9999994e-01]
 ...
 [8.5122537e-10 9.9999994e-01]
 [9.5859309e-10 1.0000000e+00]
 [2.2330306e-08 1.0000000e+00]] (275258, 2)
Server evaluation complete - Accuracy: 0.9905, Loss: 0.0324
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.9826 - loss: 0.1266
 62/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9960 - loss: 0.0096 [repeated 4x across cluster]
 52/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9945 - loss: 0.0081   
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0024 [repeated 2x across cluster]
  1/143 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 1.0000 - loss: 9.2943e-04 [repeated 6x across cluster]
 30/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9987 - loss: 0.0045  [repeated 12x across cluster]
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9975 - loss: 0.0059 [repeated 43x across cluster]
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accur

(ClientAppActor pid=11967) 
(ClientAppActor pid=11967)         
(ClientAppActor pid=11967) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=11967)             This is a deprecated feature. It will be removed
(ClientAppActor pid=11967)             entirely in future versions of Flower.
(ClientAppActor pid=11967) /Users/user/Library/Python/3.9/lib/python/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
(ClientAppActor pid=11967)   super().__init__(**kwargs)
(ClientAppActor pid=11969) /Users/user/Library/Python/3.9/lib/python/site-packages/tensorflow/python/framework/tensor_util.py:522

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8528 - loss: 0.4291 
(ClientAppActor pid=11969) Client 8 evaluation complete - Accuracy: 0.861111, Loss: 0.437581


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 310.97s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.029984190733902894
INFO :      		round 2: 0.024200499123510192
INFO :      		round 3: 0.0367810641950551
INFO :      		round 4: 0.021766862863142958
INFO :      		round 5: 0.02871871732253206
INFO :      		round 6: 0.026786555536091328
INFO :      		round 7: 0.02019612396127909
INFO :      		round 8: 0.030828221768914633
INFO :      		round 9: 0.017687762904192352
INFO :      		round 10: 0.01865598860222157
INFO :      	History (loss, centralized):
INFO :      		round 0: 1.2993907928466797
INFO :      		round 1: 0.029791992157697678
INFO :      		round 2: 0.02541358768939972
INFO :      		round 3: 0.027668915688991547
INFO :      		round 4: 0.026309898123145103
INFO :      		round 5: 0.02862519584596157
INFO :      		round 6: 0.029197068884968758
INFO :      		round 7: 0.03

Total time taken:  0:05:57.278023
 SIMULATION COMPLETE. Method = ONE_CLASS - Binary (2) Classifier
Number of Clients = 10

CPU times: user 4min 39s, sys: 33.5 s, total: 5min 12s
Wall time: 5min 57s


In [ ]:
def save_federated_model_to_pkl(server_model, scaler, output_dir="PKL"):
    """
    Lưu model federated learning đã train thành file pkl
    Thêm function này vào cuối notebook Federated Learning
    """
    import os
    import pickle
    import tensorflow as tf
    
    print("Saving federated learning model to PKL files...")
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Save the TensorFlow model
    model_path = os.path.join(output_dir, "federated_model.h5")
    server_model.save(model_path)
    print(f"TensorFlow model saved to: {model_path}")
    
    # Save the scaler
    scaler_path = os.path.join(output_dir, "scaler.pkl")
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)
    print(f"Scaler saved to: {scaler_path}")
    
    # Save model metadata
    metadata = {
        'model_type': 'TensorFlow_Federated',
        'input_shape': server_model.input_shape,
        'output_shape': server_model.output_shape,
        'num_classes': server_model.output_shape[-1],
        'feature_columns': X_columns,
        'label_column': y_column,
        'class_mappings': {
            'dict_2_classes': dict_2_classes,
            'dict_8_classes': dict_8_classes,
            'dict_34_classes': dict_34_classes
        },
        'training_config': {
            'method': METHOD,
            'num_clients': NUM_OF_CLIENTS,
            'num_rounds': NUM_OF_ROUNDS,
            'class_size': class_size
        }
    }
    
    metadata_path = os.path.join(output_dir, "federated_model_metadata.pkl")
    with open(metadata_path, 'wb') as f:
        pickle.dump(metadata, f)
    print(f"Model metadata saved to: {metadata_path}")
    
    print(f"\n=== Federated Learning Model Saved Successfully ===")
    print(f"Model files saved in: {os.path.abspath(output_dir)}")
    print("You can now use the model for attack detection!")
    
    return model_path, scaler_path, metadata_path